# Aufgabenstellung 1: Erstellen eines Prognosemodells des Kreditkartenzahlungsverkehr für Online-Einkäufe

## Beschreibung der Fallstudie:
Bereits an Deinem allerersten Tag als Data Scientist bei einem der weltweit größten Einzelhandelsunternehmen wirst Du zu einem Treffen mit Experten für Onlinezahlungen eingeladen, die Dich um Unterstützung bitten. Im letzten Jahr war die Ausfallsrate an Online-Kreditkartenzahlungen besonders hoch. Wegen dieser vielen fehlgeschlagenen Online-Überweisungen verliert das Unternehmen einerseits sehr viel Geld und andererseits werden die Kunden zunehmend unzufrieden mit dem Online-Shop des Unternehmens. Online-Kreditkartenzahlungen werden mithilfe sogenannter Zahlungsdienstleister, abgekürzt als „PSPs“ (=payments service providers), durchgeführt. Dein neuer Arbeitgeber hat mit vier verschiedenen Zahlungsdienstleistern Verträge abgeschlossen und muss für jede einzelne Überweisung Servicegebühren an diese Unternehmen zahlen. Die Logik, welcher PSP für eine bestimmte Überweisung am geeignetsten ist, basiert aktuell auf einem fixen Regelwerk und wird manuell durchgeführt. Die Entscheidungsträger innerhalb des Fachbereichs für Online- Zahlungen sind aber der Überzeugung, dass ein Prognosemodell zu besseren Entscheidungen, als ein fixes, manuelles Regelwerk, führen kann.

## Ziel des Projekts:
Unterstütze den Fachbereich für Online-Zahlungen durch ein Prognosemodell, um die Zuweisung einer Kreditkartenzahlung zu einem PSP zu automatisieren. Das Modell soll  einerseits die Erfolgsrate der Transaktionen erhöhen und andererseits die Transaktionskosten geringhalten.

## Datensatz:
Der Datensatz und weiterführende Informationen aus dem Fachbereich (Name der Zahlungsdienstleister („PSPs“), Transaktionsgebühren) sind in einem separaten zip-Dokument hinterlegt, das zum Kursmaterial angehängt wird.

Daten : PSP_Jan_Feb_2019.xlsx - Liste an Kreditkartentransaktionen der DACH Länder (Deutschland, Österreich, Schweiz)

## Detaillierte Beschreibung der Aufgabe:
Die Aufgabe besteht sowohl aus einem Programmierteil als auch aus konzeptionellen Teilaufgaben. Hier folgt eine detaillierte Beschreibung an offenen Fragen, die im finalen Dokument beantwortet werden sollen:
* Organisiere das Projekt mithilfe der CRISP-DM oder der MS Team Data Science Methode. Mache einen Vorschlag, wie die Ordnerstruktur eines Git-Repositories für das Projekt aufgebaut werden soll. Beachte, dass Du den finalen Code des Projekts nicht nach dieser Ordnerstruktur aufbauen musst.
* Beurteile die Qualität des zur Verfügung gestellten Datensatzes. Bereite Deine Erkenntnisse so auf und visualisiere sie so, dass Businesspartner in einer klaren und einfachen Weise die wichtigen Zusammenhänge verstehen können.
* Stelle ein erstes Basismodell (ein sogenanntes Baseline-Modell) auf, sowie ein präzises Vorhersagemodell, das den Businessanforderungen genügt, nämlich die Erfolgsrate der Kreditkartenzahlungen zu erhöhen und gleichzeitig die Transaktionskosten gering zu halten.
* Damit die Businesspartner Vertrauen in Dein neues Modell entwickeln, solltest du die Wichtigkeit der einzelnen erklärenden Variablen diskutieren und die Modellresultate so interpretierbar wie möglich gestalten. Außerdem ist eine detaillierte Fehleranalyse sehr wichtig, damit die Businesspartner auch die Schwachstellen Deiner Herangehensweise verstehen.
* Im letzten Schritt des Projekts soll ein Vorschlag unterbreitet werden, wie Dein Modell in die tägliche Arbeit des Fachbereichs eingebunden werden kann, beispielsweise wie eine graphische Benutzeroberfläche (GUI) aussehen könnte.
Aufgabenstellung

## Weiterführende Informationen

### Spaltenbeschreibung

• **tmsp**: Zeitstempel der Überweisung/Transaktion

• **country**: Land der Überweisung

• **amount**: Überweisungsbetrag

• **success**: wenn “1”, dann ist die Überweisung erfolgreich

• **PSP**: Name des Zahlungsdienstleisters (PSP = payments service provider)

• **3D_secured**: wenn “1”, dann ist der Kunde 3D-identifiziert (dadurch eine noch sicherere Online-Kreditkartenzahlung)

• **card**: Kreditkartenanbieter (Master, Visa, Diners)

### Kosten

| Name       | Gebühr erfolgreiche Transaktionen | Gebühr fehlgeschlagene Transaktionen |
| ---------- | --------------------------------- | ------------------------------------ |
| Moneycard  | 5 €                               | 2 €                                  |
| Goldcard   | 10 €                              | 5 €                                  |
| UK\_Card   | 3 €                               | 1 €                                  |
| Simplecard | 1 €                               | 0,5 €                                |


### Weiterführende Informationen von geschäftlicher Seite

Oftmals scheitern Überweisungen beim ersten Mal. Deshalb versuchen viele Kunden, ein und dieselbe Überweisung öfters zu tätigen. Wenn zwei Überweisungen in derselben Minute, aus demselben Land und mit demselben Überweisungsbetrag stattfinden, dann ist (für eine angemessene Anzahl an Überweisungsversuchen) davon auszugehen, dass es sich um denselben Zahlungsversuch für einen Einkauf handelt. Berücksichtige beim Entwickeln eines Machine-Learning Modells diesen Fall von mehreren Zahlungsversuchen für denselben Einkauf!

# Imports

In [2]:
# Standardbibliothek
import os
from pathlib import Path

# Drittanbieter-Pakete
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from holidays import Germany

from sklearn.preprocessing import MinMaxScaler

# scikit-learn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


# Daten untersuchen

In [3]:
raw_path   = Path(r"../Data/PSP_Jan_Feb_2019.xlsx")
df = pd.read_excel(raw_path)

In [4]:
# Zeilen und Spalten
print(f"Der Dataframe hat {df.shape[1]} Spalten und {df.shape[0]} Zeilen.\n")

# Vorschau
preview = tabulate(df.head(3), headers="keys", tablefmt="github", showindex=False)
print(f"der Dataframe sieht in den ersten drei Zeilen wie folgt aus :\n {preview}. \n")


# Duplikate - Index unnamed : 0 weglassen, um alle Duplikate in den relevanten Zeilen zu prüfen
print(f"Der Dataframe komplett hat folgende Anzahl an Duplikaten: {df.duplicated().sum()}.") 
df_no_index = df.drop(columns=["Unnamed: 0"], errors="ignore")
dup_count = df_no_index.duplicated().sum()
print(f"Der DataFrame ohne 'Unnamed: 0' hat folgende Anzahl an Duplikaten: {dup_count}.")
subset_cols = [c for c in df.columns if c != "Unnamed: 0"]
df = df.drop_duplicates(subset=subset_cols, keep="first").reset_index(drop=True)
dup_count_after = df.duplicated(subset=subset_cols).sum()
print(f"Nach Entfernen der Duplikate: {dup_count_after} Duplikate verbleibend.")
print(f"Neuer DataFrame hat {df.shape[0]} Zeilen.")

# Fehlende Werte
print(f"Die Anzahl der fehlenden Werte entsprint: {df.isna().sum().sum()}.\n")



Der Dataframe hat 8 Spalten und 50410 Zeilen.

der Dataframe sieht in den ersten drei Zeilen wie folgt aus :
 |   Unnamed: 0 | tmsp                | country   |   amount |   success | PSP     |   3D_secured | card   |
|--------------|---------------------|-----------|----------|-----------|---------|--------------|--------|
|            0 | 2019-01-01 00:01:11 | Germany   |       89 |         0 | UK_Card |            0 | Visa   |
|            1 | 2019-01-01 00:01:17 | Germany   |       89 |         1 | UK_Card |            0 | Visa   |
|            2 | 2019-01-01 00:02:49 | Germany   |      238 |         0 | UK_Card |            1 | Diners |. 

Der Dataframe komplett hat folgende Anzahl an Duplikaten: 0.
Der DataFrame ohne 'Unnamed: 0' hat folgende Anzahl an Duplikaten: 81.
Nach Entfernen der Duplikate: 0 Duplikate verbleibend.
Neuer DataFrame hat 50329 Zeilen.
Die Anzahl der fehlenden Werte entsprint: 0.



- Es gibt die Spalten Unnamed: 0 , tmsp, country, amount, success, PSP,3D_secured, card
- Es gab 81 Duplikate, welche entfernt wurden.
- Es gibt keine fehlende Werte

In [5]:
# Datentypen betrachten und prüfen, ob sie mit denen übereinstimmen, die für Sie Sinne rgeben

print(f"Die Datentypen der Spalten sind wie folgt: \n" )
df.info()

Die Datentypen der Spalten sind wie folgt: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50329 entries, 0 to 50328
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  50329 non-null  int64         
 1   tmsp        50329 non-null  datetime64[ns]
 2   country     50329 non-null  object        
 3   amount      50329 non-null  int64         
 4   success     50329 non-null  int64         
 5   PSP         50329 non-null  object        
 6   3D_secured  50329 non-null  int64         
 7   card        50329 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 3.1+ MB


- Die Datentypen passen zu den erwarteten

In [6]:
# Amount nach Verteilung und Ausreißern anschauen
print(f"\n Die statistischen Werte der Spalte amount sind : \n {df['amount'].describe().round(2)}.\n")


 Die statistischen Werte der Spalte amount sind : 
 count    50329.00
mean       202.38
std         96.26
min          6.00
25%        133.00
50%        201.00
75%        269.00
max        630.00
Name: amount, dtype: float64.



Die Verteilung der Transaktionsbeträge ist weitgehend symmetrisch, da der Mittelwert (202.38€) und der Median (201.00€) fast identisch sind. Die Daten weisen jedoch mit einer Standardabweichung von 96.26€ eine erhebliche Streuung auf, wobei die Hälfte aller Transaktionen in einem Bereich zwischen 133€ und 269€ liegt. Die Beträge decken eine große Bandbreite von 6€ bis zu einem Maximum von 630€ ab.

In [7]:
# Kategorien betrachten und schauen, ob irgendwo ein Eintrag ist, der so nicht sinnvoll ist
ausgabe_bis_cat = 30
non_numeric_columns = ["country", "success", "PSP", "3D_secured", "card"]  

for col in non_numeric_columns:
    if col not in df.columns:
        print(f"Spalte '{col}' nicht vorhanden.\n")
        continue

    total = df[col].notna().sum()
    vc = df[col].value_counts(dropna=True)

    if len(vc) <= ausgabe_bis_cat:
        print(f"Unique Einträge in '{col}': {len(vc)}")
        table = [(idx, f"{cnt / total * 100:.2f}%") for idx, cnt in vc.items()]
        print(tabulate(table, headers=[col, "Anteil"], tablefmt="github"))
        print()
    else:
        print(f"Unique Einträge in '{col}': {len(vc)}\n")


Unique Einträge in 'country': 3
| country     | Anteil   |
|-------------|----------|
| Germany     | 59.97%   |
| Switzerland | 20.51%   |
| Austria     | 19.51%   |

Unique Einträge in 'success': 2
|   success | Anteil   |
|-----------|----------|
|         0 | 79.68%   |
|         1 | 20.32%   |

Unique Einträge in 'PSP': 4
| PSP        | Anteil   |
|------------|----------|
| UK_Card    | 52.43%   |
| Simplecard | 24.72%   |
| Moneycard  | 16.48%   |
| Goldcard   | 6.37%    |

Unique Einträge in '3D_secured': 2
|   3D_secured | Anteil   |
|--------------|----------|
|            0 | 76.17%   |
|            1 | 23.83%   |

Unique Einträge in 'card': 3
| card   | Anteil   |
|--------|----------|
| Master | 57.52%   |
| Visa   | 23.10%   |
| Diners | 19.38%   |



Die Ausprägungen aller kategorialen Spalten sind durchweg plausibel und entsprechen den Erwartungen.

In [8]:
# Datetime Spalte betrachten, ob die Werte den erwarteten entsprechen und ob es ungewöhnliche Verteilungen gibt Sicherstellen, 
# dass die Spalte wirklich Datetime ist
datetime_cols = ["tmsp"]                    

for col in datetime_cols:
    print(f"\n--- Analyse für Datetime-Spalte: {col} ---")
    
    series = df[col].dropna()
    total   = series.size                 

    def pct(counts):
        return (counts / total * 100).round(2)  

    print("\nAnteil pro Jahr (%):")
    print(pct(series.dt.year.value_counts().sort_index()).astype(str) + " %")

    print("\nAnteil pro Monat (%):")
    print(pct(series.dt.month.value_counts().sort_index()).astype(str) + " %")

    print("\nAnteil pro Wochentag (%):")
    print(pct(series.dt.dayofweek.value_counts().sort_index()).astype(str) + " %")

    if series.dt.hour.notna().any():
        print("\nAnteil pro Stunde (0-23) (%):")
        print(pct(series.dt.hour.value_counts().sort_index()).astype(str) + " %")




--- Analyse für Datetime-Spalte: tmsp ---

Anteil pro Jahr (%):
tmsp
2019    100.0 %
Name: count, dtype: object

Anteil pro Monat (%):
tmsp
1    52.2 %
2    47.8 %
Name: count, dtype: object

Anteil pro Wochentag (%):
tmsp
0    14.49 %
1    17.18 %
2    16.26 %
3    15.47 %
4    13.68 %
5    12.36 %
6    10.55 %
Name: count, dtype: object

Anteil pro Stunde (0-23) (%):
tmsp
0     4.13 %
1     4.15 %
2     4.23 %
3     4.33 %
4     4.03 %
5     4.23 %
6      4.1 %
7     4.14 %
8      4.1 %
9     4.12 %
10    4.25 %
11    4.16 %
12    4.15 %
13    4.25 %
14    4.08 %
15    4.18 %
16    4.27 %
17    4.21 %
18    4.04 %
19     4.1 %
20    4.33 %
21    4.21 %
22    4.08 %
23    4.14 %
Name: count, dtype: object


Die Analyse der Zeitstempel-Spalte `tmsp` zeigt, dass alle Transaktionen aus dem Jahr 2019 stammen und sich relativ gleichmäßig auf die Monate Januar (52,2 %) und Februar (47,8 %) verteilen. Innerhalb der Woche ist die Transaktionsaktivität an den Werktagen höher, mit einem Höhepunkt am Dienstag (17,2 %), und nimmt zum Wochenende hin, insbesondere am Sonntag (10,6 %), deutlich ab. Über den Tagesverlauf sind die Transaktionen hingegen sehr gleichmäßig verteilt, da jede Stunde einen annähernd gleichen Anteil von etwa 4 % am Gesamtvolumen hat.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50329 entries, 0 to 50328
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  50329 non-null  int64         
 1   tmsp        50329 non-null  datetime64[ns]
 2   country     50329 non-null  object        
 3   amount      50329 non-null  int64         
 4   success     50329 non-null  int64         
 5   PSP         50329 non-null  object        
 6   3D_secured  50329 non-null  int64         
 7   card        50329 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 3.1+ MB


# Dataframe ohne Duplikate speichern

In [10]:
this_dir = Path.cwd()         #/data_analysis
data_dir   = this_dir / ".." / "Data"               # …/Data   
data_dir   = data_dir.resolve()                     # in absoluten Pfad umwandeln
pkl_path   = data_dir / "df_original.pkl"     # /Data/df_oridinal.pkl

df.to_pickle(pkl_path)

# Überprüfen
print("Exists:", os.path.exists(pkl_path))
#print(pkl_path)


Exists: True
